# Set up a supercell of a grain boundary and determine the energy

## Install lammmps software


In [ ]:
# Install lammps
!pip install lammps[mpi]
# Install package to assist in parsing lammps log files
!pip install lammps-logfile

## Mount your Google drive
...and change to the correct directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
cd /content/drive/MyDrive/cdt_training/MaterialsModellingPrimer/04-GrainBoundary

## Load some modules 

In [2]:
import numpy as np
import os
import shutil
import subprocess
from contextlib import chdir
import lammps_logfile
import matplotlib.pyplot as plt
%matplotlib inline

## Define some useful functions

In [3]:
# Define a function to replace text in base input file
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

## Run the simulations

In [ ]:
pair_styles = ['eam/alloy', 'eam/fs', 'eam/alloy', 'eam/alloy', 'eam/alloy', 'eam/alloy']
potential_files = ['Pot1.set', 'Pot2.eam.fs', 'Pot3.eam.alloy', 'Pot4.eam.alloy', 'Pot5.eam.alloy', 'Pot6.eam.fs']

potential_index =  

if True:
    potential_file = potential_files[potential_index]
    pair_style = pair_styles[potential_index]

    # ------------- Set up a folder for the simulation
    path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/'
    if not os.path.exists(path):
        print(path)
        os.makedirs(path, mode=0o777)

    # ------------- Copy the grain boundary structure
    shutil.copy2('lammps.txt', path)

    # Create  a folder for simulation and copy in the empirical potential file
    shutil.copy2('../Potentials/' + potential_file, path)

    # Define values to replace in base input file
    replacements = {
        'POTENTIAL':potential_file,
        'PAIRSTYLE':pair_style
        }
        
    # Create a lammps file and use base file with string replacements to create unique variant
    inputLammpsFile = 'in_base.lmps'
    outputLammpsFile = path+'in.lmps'        
    finLammps = open(inputLammpsFile, 'r').read()
    foutLammps = open(outputLammpsFile, 'w')
    out = replace_all(finLammps, replacements)
    foutLammps.write(out)
    foutLammps.close()

    lammps_executable = 'lmp'
    lammps_command = '-in in.lmps'

    with chdir(path):
        os.system(lammps_executable + ' ' + lammps_command)

## Check the results from the simulations

These cells are for use after the simulation has run, to examine the output.

Read in data from the logfile and calculate the grain boundary energy.

In [ ]:
ev = 1.602E-19 #(J)
A = 1E-10 #(m)

colwidth = 24

output_path_bulk = '../01-LatticeParameters/Simulations/'
data_bulk = np.loadtxt(output_path_bulk + 'results.txt', skiprows=1)

natoms = 19120

# Loop over potentials
if True:
    potential_file = potential_files[potential_index]
    path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/'
    log = lammps_logfile.File(path + 'log.lammps')

    step = log.get("Step")
    pe = log.get("PotEng")
    ly = log.get("Ly")
    lz = log.get("Lz")
    pressure = log.get("Pxx")

    E0 = data_bulk[potential_index,2]
    Egb = (pe[-1] - natoms*E0)/ly[-1]/lz[-1]

    print(f'Our estimate of the grain boundary energy is {Egb:0.4f} eV/A^2')  
    


